In [ ]:
import os
import openai

In [ ]:
from typing import List
from pydantic import BaseModel, Field

In [ ]:
class User:
    def __init__(self, name: str, age:int, email:str) -> None:
        self.name = name
        self.age = age
        self.email = email

In [ ]:
user = User(name="Dev", age="35", email="devpatel212@gmail.com")

In [ ]:
user.age

In [ ]:
class pUser(BaseModel):
    name: str
    age:int
    email:str

In [ ]:
foo_p = pUser(name="Dev", age="34", email="devpatel212@gmail.com")

In [ ]:
class Class(BaseModel):
    students: List[pUser]

In [ ]:
pObj = Class(students=[pUser(name="Mandvi",age=30,email="mandvipatel@gmail.com")])

In [ ]:
pObj.model_dump()

In [ ]:
class WeatherSearch(BaseModel):
    '''Call this with the airport code to get the weather at airport'''
    airpodCode: str = Field(description="airpod code to get weather for")

In [ ]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
weather_search = convert_pydantic_to_openai_function(WeatherSearch)

In [ ]:
weather_search

In [ ]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI()
model.invoke("What is weather of SF today?", functions=[weather_search])

In [ ]:
from typing import Optional
from langchain.prompts import ChatPromptTemplate
from langchain.chains.openai_functions import (
     create_openai_fn_chain,
    create_structured_output_chain,
    create_openai_fn_runnable,
    create_structured_output_runnable,
)


class Person(BaseModel):
    '''Identifying information about a person'''
    name: str
    age: str
    fav_food: Optional[str] = Field(None, description="The person's favourite food")

In [ ]:
llm = ChatOpenAI()
prompt =  ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world class algorithm for extracting information in structured formats.",
        ),
        (
            "human",
            "Use the given format to extract information from the following input: {input}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

person = convert_pydantic_to_openai_function(Person)
person


model = create_structured_output_runnable(person, llm, prompt)

# model.invoke({"input": "Sally is 13"})

In [ ]:
from typing import Optional
from langchain.pydantic_v1 import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

from langchain.chains.openai_functions import (
     create_openai_fn_chain,
    create_structured_output_chain,
    create_openai_fn_runnable,
    create_structured_output_runnable,
)


class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")

In [ ]:
# If we pass in a model explicitly, we need to make sure it supports the OpenAI function-calling API.
llm = ChatOpenAI(temperature=0,verbose=True)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world class algorithm for extracting information in structured formats.",
        ),
        (
            "human",
            "Use the given format to extract information from the following input: {input}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

chain = create_structured_output_chain(Person, llm, prompt)
resp = chain.run({"input": "Selly is 13"})
resp

In [25]:
from pydantic import BaseModel
from enum import Enum
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableParallel
from typing import Optional, List
from langchain.globals import set_verbose

set_verbose(True)

In [26]:
llm = ChatOpenAI( temperature=0, verbose=True)
   
    
class GeneralFeedbackTitle(str, Enum):
    GENERAL_FEEDBACK: str  = "General Feedback"
    SUGGESTED_ANSWER:str = "Suggested Answer"

class GeneralFeedbackItem(BaseModel):
    title: GeneralFeedbackTitle
    content: str
    
class GeneralFeedback(BaseModel):
    feedback: List[GeneralFeedbackItem]
    
    
class FeedbackTitle(str, Enum):
    PEEL_FEEDBACK:str = "PEEL Feedback"


class PeelFeedbackItem(BaseModel):
        Point:str = "PEEL Feedback"
        Explanation:str = "Explation Feedback"
        Example:str = "Example Feedback"
        Link: str = "Link Feedback"

class FeedbackItem(BaseModel):
    title: FeedbackTitle
    content: PeelFeedbackItem
    
class Feedback(BaseModel):
    feedback: List[FeedbackItem]
    
    

In [24]:
general_chain = ChatPromptTemplate.from_template("""I will give you a question prompt that is based on an example of the picture description part of a Primary 6 English oral examination in Singapore.
    I will provide a model answer and a student's answer. I would like you to give appropriate feedback on what is lacking in the student's answer and how the student's answer can be improved.
    Your content, which must be child appropriate, should sound encouraging to the student. Address the student as first person (i.e. do not say 'the student). Specifically,
    I would like you to give your response in 3 parts: 
    1) general feedback first, starting with an encouraging phrase like 'good try', highlighting any errors in grammar or vocabulary and show how it can be improved, 
    2) providing a  suggested answer, in one paragraph (about 150 words) speech, enhance upon the student's answer.
    The suggested answer is replied as if you are the student.
    Question prompt: {question_prompt}
    Model answer (in PEEL format): {model_answer}
    Example student answer: {student_answer}
    """)
chain1 = create_structured_output_chain(GeneralFeedback, llm, general_chain)

peel_chain = ChatPromptTemplate.from_template("""I will give you a question prompt that is based on an example of the picture description part of a Primary 6 English oral examination in Singapore.
    I will provide a model answer and a student's answer. I would like you to give appropriate feedback on what is lacking in the student's answer and how the student's answer can be improved.
    Your content, which must be child appropriate, should sound encouraging to the student. Address the student as first person (i.e. do not say 'the student). Specifically,
    I would like you to give your response in 3 parts: 
    1) focusing on PEEL feedback, specifying if the student fulfilled each aspect.
    List in the bullet points: 'Point', 'Explanation', 'Example, 'Link') and start each bullet point as a  new paragraph;
    The suggested answer is replied as if you are the student.
    Question prompt: {question_prompt}
    Model answer (in PEEL format): {model_answer}
    Example student answer: {student_answer}
    """) 

chain2 = create_structured_output_chain(Feedback, llm, peel_chain)
    
map_chain = RunnableParallel(generalFeedback=chain1, peelFeedback=chain2)
    
resp = map_chain.invoke({"question_prompt": """We must teach Singaporeans more about the marine life found in Singapore. Do you agree? Why?""", 
                         "model_answer":"""Yes. I agree. I don’t know very much about the marine life in Singapore. In fact, I don’t think any of my classmates and friends do. We learn about natural habitats and ecosystems of places from the textbook but it does not tell us much about the local marine life. I am certain there are some native marine creatures, but I can’t name them nor can I identify them. We should be taught more about our natural heritage, so that we can treasure our environment more. Unless we learn about the value of our environment, we won’t take care of it. For instance, when my class went on a field trip to MacRitchie Reservoir last year, I saw tortoises in the water. I hear that they are released into the water but I know that our actions will impact the environment and may upset the ecosystem. We should teach Singaporeans about how our actions can affect our environment.""",
                         "student_answer":"""Yes we should take care of out ocean"""})

RuntimeError: no validator found for <class '__main__.GeneralFeedback'>, see `arbitrary_types_allowed` in Config